# ES6 规范中的词法环境与作用域、闭包的关系


## 词法环境（Lexical Environment）结构

点击查看 [ES6 规范对词法环境（Lexical Environment）的描述](https://262.ecma-international.org/6.0/#sec-lexical-environments)。

在 ES6 规范中，词法环境是变量和函数的静态上下文。结构如下：

- 环境记录（存储变量和函数）
- 对外部词法环境的引用

**词法环境有三种**

- 全局词法环境
  全局词法环境是 JS 脚本在执行时创建的最外层词法环境。在全局范围内声明的变量和函数都存在于全局词法环境的环境记录。
  在整个 JS 脚本中只有一个全局词法环境，并且它存在于 JS 脚本的整个生命周期。
  全局词法环境的环境记录与全局对象（window 或 global）绑定，在整个 JS 脚本中可访问。

- 函数词法环境
  创建于每次函数被调用。
  函数的参数、局部变量和内部函数都存在于函数词法环境的环境记录。
  函数词法环境的生命周期通常与函数的执行周期一致，即函数执行完，对应的函数词法环境被收回。
  但有一例外，就是如果一个函数创建了闭包，那么即使这个函数执行完，它的词法环境依然被引用。
  何为闭包：闭包就是一个函数把自己的词法环境暴露出去。

- 模块词法环境
  模块词法环境创建于 ES6 模块的加载执行，模块词法环境环境记录内的变量和函数默认是私有的，除非通过 export 关键字导出。

环境记录有三种类型：

- 声明性环境记录：存储由 variable, constant, let, class, module, import, function declarations 等声明的标识符。
  - 函数环境记录：存储函数声明和函数的参数信息，如果该函数不是 ArrowFunction，则提供 this 绑定。
  - 模块环境记录：存储模块的导入和导出声明。
- 对象环境记录：存储由全局对象（如 window 或 global）。
- 全局环境记录：声明性环境记录 + 对象环境记录。


## 作用域


## 闭包

函数词法环境外部变量的引用（闭包）。
